In [1]:
import pandas as pd
import numpy as np
from usfull_tools import load_DS
from catboost import Pool, CatBoostRegressor, CatBoostClassifier
from sklearn.model_selection import train_test_split, cross_val_score
pd.options.display.max_columns = None
%matplotlib inline

from set_vars import KAGGLE_PREFIX, debug_mode, KAGGLE_DIR, target_column, target_type, loss_function, custom_metric

train, test = load_DS(debug_mode, KAGGLE_DIR, KAGGLE_PREFIX, '_prepare.csv')
del test

cols = pd.read_csv(KAGGLE_DIR + KAGGLE_PREFIX + '_important_columns.csv')

X_train = train[cols['Feature']]
y_train = train[target_column]

X_train.head(10)

,sum_OverallQual_log_PoolQC,multiply_OverallQual_GarageCars,multiply_GrLivArea_GarageCars,division_OverallQual_YearRemodAdd,multiply_GrLivArea_FullBath,multiply_OverallQual_GarageYrBlt,multiply_OverallQual_TotalBsmtSF,division_YearRemodAdd_GrLivArea,multiply_1stFlrSF_TotRmsAbvGrd,multiply_OverallQual_log_LotArea,multiply_OverallQual_BsmtFinSF1,multiply_OverallQual_FullBath,multiply_OverallQual_TotRmsAbvGrd,sub_GarageYrBlt_log_PoolArea,sub_1stFlrSF_log_PoolArea,BldgType,multiply_OverallQual_GarageArea,GrLivArea,multiply_OpenPorchSF_MoSold,multiply_OverallQual_YearRemodAdd,multiply_TotalBsmtSF_FullBath,multiply_OverallQual_log_PoolQC,sub_BsmtFinSF1_Fireplaces,multiply_OverallQual_Fireplaces,division_TotalBsmtSF_GarageYrBlt,division_YearRemodAdd_TotRmsAbvGrd,sum_2ndFlrSF_log_PoolArea,multiply_1stFlrSF_PoolQC,multiply_FullBath_GarageArea,multiply_OverallQual_1stFlrSF,MSSubClass,multiply_YearRemodAdd_TotRmsAbvGrd,GarageFinish,SaleCondition,sub_TotalBsmtSF_HalfBath,sum_BsmtFinSF1_MoSold,OverallCond,BsmtQual,multiply_2ndFlrSF_PoolQC,division_OverallCond_GrLivArea,log_LotArea,multiply_BsmtUnfSF_PoolQC,multiply_TotalBsmtSF_PoolQC,multiply_2ndFlrSF_log_PoolQC,sub_2ndFlrSF_log_PoolArea,sum_2ndFlrSF_log_MiscVal,multiply_YearRemodAdd_FullBath,WoodDeckSF,multiply_GrLivArea_log_PoolQC,Fireplaces,division_Alley_GrLivArea,MSZoning,log_KitchenAbvGr,multiply_TotRmsAbvGrd_MiscFeature,sum_YearBuilt_log_PoolArea,sub_TotalBsmtSF_log_PoolArea,log_EnclosedPorch,BedroomAbvGr,TotRmsAbvGrd,multiply_GarageCars_PoolQC,EnclosedPorch,sub_OpenPorchSF_log_MiscVal,LandContour,1stFlrSF,division_YearBuilt_GrLivArea,multiply_GrLivArea_BedroomAbvGr,division_OpenPorchSF_MoSold,sum_YrSold_log_PoolArea,division_OverallQual_1stFlrSF,OverallQual,division_OverallQual_log_LotArea,multiply_LotFrontage_Fireplaces,sum_TotalBsmtSF_FullBath,division_BsmtFinSF1_GarageYrBlt,PavedDrive,multiply_BsmtUnfSF_GrLivArea,BsmtExposure,YearRemodAdd,sub_YearRemodAdd_log_PoolArea,LotArea,multiply_MSSubClass_MasVnrArea
0,7.695312,14.0,3420.0,0.003494,3420,14024.0,5992.0,1.170898,6848,63.28125,4944.0,14,56,2003.0,856.0,1Fam,3836.0,1710,122,14021,1712.0,4.851562,706.0,0,0.427246,250.375,854.0,856.0,1096.0,5992,60,16024,RFn,Normal,855.0,708.0,5,Gd,854.0,0.002924,9.039062,150.0,856.0,592.0,854.0,854.0,4006,0,1186.0,0,0.000585,RL,0.693359,8.0,2003.0,856.0,0.000000,3,8,2.0,0,61.000000,Lvl,856,1.170898,5130,30.500000,2008.0,0.008179,7,0.773926,0.0,858.0,0.352539,Y,256500.0,No,2003,2003.0,8450,11760.0
1,6.695312,12.0,2524.0,0.003036,2524,11856.0,7572.0,1.565430,7572,55.03125,5868.0,12,36,1976.0,1262.0,1Fam,2760.0,1262,0,11856,2524.0,4.160156,977.0,6,0.638672,329.250,0.0,1262.0,920.0,7572,20,11856,RFn,Normal,1262.0,983.0,8,Gd,0.0,0.006340,9.171875,284.0,1262.0,0.0,0.0,0.0,3952,298,875.0,1,0.000793,RL,0.693359,6.0,1976.0,1262.0,0.000000,3,6,2.0,0,0.000000,Lvl,1262,1.565430,3786,0.000000,2007.0,0.004753,6,0.654297,80.0,1264.0,0.494873,Y,358408.0,Gd,1976,1976.0,9600,0.0
2,7.695312,14.0,3572.0,0.003496,3572,14008.0,6440.0,1.121094,5520,65.31250,3402.0,14,42,2001.0,920.0,1Fam,4256.0,1786,378,14014,1840.0,4.851562,485.0,7,0.459717,333.750,866.0,920.0,1216.0,6440,60,12012,RFn,Normal,919.0,495.0,5,Gd,866.0,0.002800,9.328125,434.0,920.0,600.5,866.0,866.0,4004,0,1238.0,1,0.000560,RL,0.693359,6.0,2001.0,920.0,0.000000,3,6,2.0,0,42.000000,Lvl,920,1.120117,5358,4.667969,2008.0,0.007610,7,0.750488,68.0,922.0,0.242920,Y,775124.0,Mn,2002,2002.0,11250,9720.0
3,7.695312,21.0,5152.0,0.003553,1717,13984.0,5292.0,1.147461,6727,64.12500,1512.0,7,49,1998.0,961.0,1Fam,4496.0,1717,70,13790,756.0,4.851562,215.0,7,0.378418,281.500,756.0,961.0,642.0,6727,70,13790,Unf,Abnorml,756.0,218.0,5,TA,756.0,0.002913,9.164062,540.0,756.0,524.0,756.0,756.0,1970,0,1190.0,1,0.000582,RL,0.693359,7.0,1915.0,756.0,5.609375,3,7,3.0,272,35.000000,Lvl,961,1.115234,5151,17.500000,2006.0,0.007282,7,0.763672,60.0,757.0,0.108093,Y,927180.0,No,1970,1970.0,9550,0.0
4,8.695312,24.0,6592.0,0.004002,4396,16000.0,9160.0,0.910156,10305,76.50000,5240.0,16,72,

In [ ]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
import time


i=0
cat_features = []
for column in X_train.columns:
    if X_train[column].dtype == 'object': cat_features.append(i)
    i +=1

def hyperopt_train_test(params):
    if target_type=='binary': clf = CatBoostClassifier(**params)
    elif target_type=='interval': clf = CatBoostRegressor(**params)
    return cross_val_score(clf, X_train, y_train, cv=5).mean()


if target_type=='binary':
    space4CatBoost = {
        'random_seed': 42,
        'iterations': hp.choice('iterations', range(100, 2000, 20)),
        'depth': hp.choice('depth', {1, 2, 3, 4, 5, 6}),
        'l2_leaf_reg': hp.uniform('l2_leaf_reg', 1, 10),
        'learning_rate': hp.uniform('learning_rate', 0.0001, 1),
        'loss_function': loss_function,
        'od_type': "Iter",
        'od_wait': 100,
        'verbose': False,
        'cat_features': cat_features
    }
    best_acc = 0

elif target_type=='interval':
    space4CatBoost = {
        'random_seed': 42,
        'iterations': hp.choice('iterations', range(100, 2000, 20)),
        'depth': hp.choice('depth', {1, 2, 3, 4, 5, 6}),
        'l2_leaf_reg': hp.uniform('l2_leaf_reg', 1, 10),
        'learning_rate': hp.uniform('learning_rate', 0.0001, 1),
        'loss_function': loss_function,
        'od_type': "Iter",
        'od_wait': 100,
        'verbose': False,
        'cat_features': cat_features
    }
#     space4CatBoost = {
#         'random_seed': 42,
#         'iterations': hp.choice('iterations', range(100, 1000, 100)),
#         'depth': hp.choice('depth', {1, 2, 4, 6}),
#         'l2_leaf_reg': hp.choice('l2_leaf_reg', {1, 2, 3, 4, 6}),
#         'learning_rate': hp.uniform('learning_rate', 0.0001, 0.8),
#         'loss_function': loss_function,
#         'od_type': 'Iter',
#         'verbose': False,
#         'cat_features': cat_features
#     }
    best_acc = 9999999

best_params = {}

def f(params):
    global best_params, best_acc
    acc = hyperopt_train_test(params)
        
    if best_acc < acc and target_type=='binary' or best_acc > acc and target_type=='interval':
        best_acc = acc
        best_params = params
        print('\nNew best acc:', best_acc, params)

    if target_type=='interval':
        return {'loss': acc, 'status': STATUS_OK}
    else:
        return {'loss': -acc, 'status': STATUS_OK}

trials = Trials()

best = fmin(f, space4CatBoost, algo=tpe.suggest, max_evals=50, trials=trials)

print ('best_params:', best_params)


                                                                                                                       
New best acc:
29327.96432114744                                                                                                      
{'cat_features': (15, 32, 33, 37, 51, 62, 74, 76), 'depth': 4, 'iterations': 160, 'l2_leaf_reg': 2.4397774916351658, 'learning_rate': 0.05697921120509066, 'loss_function': 'RMSE', 'od_type': 'Iter', 'od_wait': 100, 'random_seed': 42, 'verbose': False}
                                                                                                                       
New best acc:
27622.25087197575                                                                                                      
{'cat_features': (15, 32, 33, 37, 51, 62, 74, 76), 'depth': 1, 'iterations': 1720, 'l2_leaf_reg': 6.283273392794612, 'learning_rate': 0.3381804970576962, 'loss_function': 'RMSE', 'od_type': 'Iter', 'od_wait': 100, 'random_seed': 42, 'verbos

In [ ]:
if target_type == 'binary': model = CatBoostClassifier(iterations = best_params['iterations'], depth = best_params['depth'], 
                               l2_leaf_reg = best_params['l2_leaf_reg'], learning_rate = best_params['learning_rate'], 
                               loss_function = loss_function, od_type = 'Iter', verbose = False)
    
elif target_type == 'interval': model = CatBoostRegressor(iterations = best_params['iterations'], depth = best_params['depth'], 
                               l2_leaf_reg = best_params['l2_leaf_reg'], learning_rate = best_params['learning_rate'], 
                               loss_function = loss_function, od_type = 'Iter', verbose = False)

model.fit(X_train, y_train, cat_features = cat_features, verbose = False, plot = True)

In [ ]:
import pickle
# save the classifier
with open(KAGGLE_DIR + KAGGLE_PREFIX + 'classifier.pkl', 'wb') as fid:
    pickle.dump(model, fid)

In [ ]:
print ('trials:')
for trial in trials.trials:
    print (trial)

# 